## Transformers concepts

### 3 Types of Classes:

1. model class --> torch.nn.Modules (e.g. BertModel)
2. configuration classes --> Stores all parameters required to build a model e.g. BertConfig
3. tokenizer classes --> store vocabulary & provide methods for encoding / decoding strings in lists of token embeddings indices to be fed into a model e.g. BertTokenizer

In [4]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import logging
#logging.basicConfig(level=logging.INFO)

In [6]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

INFO:filelock:Lock 139862080759624 acquired on /home/nora/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache or force_download set to True, downloading to /home/nora/.cache/torch/transformers/tmptpa8bh31


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt in cache at /home/nora/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:transformers.file_utils:creating metadata file for /home/nora/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:filelock:Lock 139862080759624 released on /home/nora/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/nora/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a

In [177]:
# Tokenize input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', 'henson', 'was', 'a', 'puppet', '##eer', '[SEP]']


In [178]:
# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 11
tokenized_text[masked_index] = '[MASK]'
tokenized_text

['[CLS]',
 'who',
 'was',
 'jim',
 'henson',
 '?',
 '[SEP]',
 'jim',
 'henson',
 'was',
 'a',
 '[MASK]',
 '##eer',
 '[SEP]']

In [179]:
# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)

[101, 2040, 2001, 3958, 27227, 1029, 102, 3958, 27227, 2001, 1037, 103, 11510, 102]


In [180]:
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

In [181]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

## Use the BertModel to encode inputs in hidden-states

In [182]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/nora/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
INFO:transformers.configuration_utils:Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_

In [183]:
# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [184]:
# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [185]:
!export CUDA_VISIBLE_DEVICES=1

In [186]:
# Predict hidden states features for each layer
with torch.no_grad():
    # See the models docstrings for the detail of the inputs
    outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    # Transformers models always output tuples.
    # See the models docstrings for the detail of all the outputs
    # In our case, the first element is the hidden state of the last layer of the Bert model
    encoded_layers = outputs[0]
# We have encoded our input sequence in a FloatTensor of shape (batch size, sequence length, model hidden dimension)
assert tuple(encoded_layers.shape) == (1, len(indexed_tokens), model.config.hidden_size)
encoded_layers

tensor([[[-0.5866,  0.0946, -0.7525,  ..., -1.0007,  0.7824,  0.5625],
         [-0.9850,  0.0090, -0.0484,  ...,  0.1264,  0.0669, -0.3493],
         [ 0.1395, -0.6434, -0.6270,  ..., -0.0759,  0.4486,  0.2664],
         ...,
         [ 0.2402,  0.0180,  0.2846,  ...,  0.3656,  0.3144, -0.4937],
         [-0.1459,  0.1321,  0.3220,  ..., -0.1364,  0.1667, -0.7756],
         [ 0.8121,  0.2787, -0.3609,  ...,  0.2309, -0.3461, -0.3172]]],
       device='cuda:0')

## Use BertForMaskedML to predict masked token

In [187]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/nora/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
INFO:transformers.configuration_utils:Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [188]:
# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [189]:
# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    predictions = outputs[0]

In [190]:
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

In [191]:
predicted_token

'puppet'

# QA

In [192]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

In [193]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

INFO:filelock:Lock 139862758082768 acquired on /home/nora/.cache/torch/transformers/00bf8b18639562831ff9d22825e646f00f34a6976c6ed0ed1c4b35d334bb7a51.84dc10193e066c2b505f965e0ba0751ef60dba0afccfad2e23acd97eb7feec5e.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-config.json not found in cache or force_download set to True, downloading to /home/nora/.cache/torch/transformers/tmp9mhgbjde


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-config.json in cache at /home/nora/.cache/torch/transformers/00bf8b18639562831ff9d22825e646f00f34a6976c6ed0ed1c4b35d334bb7a51.84dc10193e066c2b505f965e0ba0751ef60dba0afccfad2e23acd97eb7feec5e
INFO:transformers.file_utils:creating metadata file for /home/nora/.cache/torch/transformers/00bf8b18639562831ff9d22825e646f00f34a6976c6ed0ed1c4b35d334bb7a51.84dc10193e066c2b505f965e0ba0751ef60dba0afccfad2e23acd97eb7feec5e
INFO:filelock:Lock 139862758082768 released on /home/nora/.cache/torch/transformers/00bf8b18639562831ff9d22825e646f00f34a6976c6ed0ed1c4b35d334bb7a51.84dc10193e066c2b505f965e0ba0751ef60dba0afccfad2e23acd97eb7feec5e.lock
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-config.json from cache at /home/nora/.cache/torch/tra

INFO:filelock:Lock 139862763106600 acquired on /home/nora/.cache/torch/transformers/425869b134694b14f3fd06adbee0d84e62abc76097792a5c6f94524239e0e8b4.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-vocab.txt not found in cache or force_download set to True, downloading to /home/nora/.cache/torch/transformers/tmpk2m8anqw


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-vocab.txt in cache at /home/nora/.cache/torch/transformers/425869b134694b14f3fd06adbee0d84e62abc76097792a5c6f94524239e0e8b4.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:transformers.file_utils:creating metadata file for /home/nora/.cache/torch/transformers/425869b134694b14f3fd06adbee0d84e62abc76097792a5c6f94524239e0e8b4.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:filelock:Lock 139862763106600 released on /home/nora/.cache/torch/transformers/425869b134694b14f3fd06adbee0d84e62abc76097792a5c6f94524239e0e8b4.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-vocab.txt from cache at /home/nora/.cache/torch/transformers/425869b13

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-config.json from cache at /home/nora/.cache/torch/transformers/00bf8b18639562831ff9d22825e646f00f34a6976c6ed0ed1c4b35d334bb7a51.84dc10193e066c2b505f965e0ba0751ef60dba0afccfad2e23acd97eb7feec5e
INFO:transformers.configuration_utils:Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LAB

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-pytorch_model.bin in cache at /home/nora/.cache/torch/transformers/ca2ac20761877486c1e2204d99653106b9adacf9a5eb18ec71b41d2dbef42103.2db7ae79c41a184c87600faabafa1369db2b16457723fd154ca3b436c4172807
INFO:transformers.file_utils:creating metadata file for /home/nora/.cache/torch/transformers/ca2ac20761877486c1e2204d99653106b9adacf9a5eb18ec71b41d2dbef42103.2db7ae79c41a184c87600faabafa1369db2b16457723fd154ca3b436c4172807
INFO:filelock:Lock 139862029095880 released on /home/nora/.cache/torch/transformers/ca2ac20761877486c1e2204d99653106b9adacf9a5eb18ec71b41d2dbef42103.2db7ae79c41a184c87600faabafa1369db2b16457723fd154ca3b436c4172807.lock
INFO:transformers.modeling_utils:loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-pytorch_model.bin from cache at /home/nora/.cache/torch/tr

In [194]:
text = r"""
James Marshall "Jimi" Hendrix was an American rock guitarist, singer, and songwriter. 
Although his mainstream career lasted only four years, he is widely regarded as one of the 
most influential guitarists in history and one of the most celebrated musicians of the 20th century.
"""

In [199]:
questions = [
    "Where is Jimi Hendirx from?",
    "Where is Jimi Hendrix from?",
    "How long did his mainstream career last?",
    "What is Jimi Hendrix' real name?",
]

In [200]:
for question in questions:
    
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer_start_scores, answer_end_scores = model(**inputs)

    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: Where is Jimi Hendirx from?
Answer: james marshall " jimi " hendrix

Question: Where is Jimi Hendrix from?
Answer: american

Question: How long did his mainstream career last?
Answer: four years

Question: What is Jimi Hendrix' real name?
Answer: james marshall " jimi " hendrix



In [202]:
from transformers import pipeline

nlp = pipeline("fill-mask")
print(nlp(f"Jimi Hendrix is a {nlp.tokenizer.mask_token} from America."))


INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilroberta-base-config.json from cache at /home/nora/.cache/torch/transformers/d52ced8fd31ba6aa311b6eeeae65178cca00ddd6333c087be4601dc46c20bd96.deeb956b92b63ef883f183df980353df2c982e37e78a00189cf9146088df28dd
INFO:transformers.configuration_utils:Model config RobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/distilroberta-base-modelcard.json in cache at /home/nora/.cache/torch/transformers/8df1a5d2cc59a6b2d440c323b3d4287c795d8b2d33d2b7f37cb7290d6ecfc70a.455d944f3d1572ab55ed579849f751cf37f303e3388980a42d94f7cd57a4e331
INFO:transformers.file_utils:creating metadata file for /home/nora/.cache/torch/transformers/8df1a5d2cc59a6b2d440c323b3d4287c795d8b2d33d2b7f37cb7290d6ecfc70a.455d944f3d1572ab55ed579849f751cf37f303e3388980a42d94f7cd57a4e331
INFO:filelock:Lock 139862029095824 released on /home/nora/.cache/torch/transformers/8df1a5d2cc59a6b2d440c323b3d4287c795d8b2d33d2b7f37cb7290d6ecfc70a.455d944f3d1572ab55ed579849f751cf37f303e3388980a42d94f7cd57a4e331.lock
INFO:transformers.modelcard:loading model card file https://s3.amazonaws.com/models.huggingface.co/bert/distilroberta-base-modelcard.json from cache at /home/nora/.cache/torch/transformers/8df1a5d2cc59a6b2d440c323b3d4287c795d8b2d33d2b7f37cb7290d6ecfc70a.45

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilroberta-base-config.json from cache at /home/nora/.cache/torch/transformers/d52ced8fd31ba6aa311b6eeeae65178cca00ddd6333c087be4601dc46c20bd96.deeb956b92b63ef883f183df980353df2c982e37e78a00189cf9146088df28dd
INFO:transformers.configuration_utils:Model config RobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm

[{'sequence': '<s> Jimi Hendrix is a musician from America.</s>', 'score': 0.28688114881515503, 'token': 9613}, {'sequence': '<s> Jimi Hendrix is a singer from America.</s>', 'score': 0.15069352090358734, 'token': 3250}, {'sequence': '<s> Jimi Hendrix is a transplant from America.</s>', 'score': 0.0895230621099472, 'token': 14067}, {'sequence': '<s> Jimi Hendrix is a rapper from America.</s>', 'score': 0.08071661740541458, 'token': 9102}, {'sequence': '<s> Jimi Hendrix is a native from America.</s>', 'score': 0.063807412981987, 'token': 3763}]


In [203]:
from transformers import pipeline

summarizer = pipeline("summarization")

INFO:filelock:Lock 139862067348424 acquired on /home/nora/.cache/torch/transformers/5f0de1d2bbb8eb1a3b69656622293b3328b06b701663a9d4109359751cb4e739.a42ba2fc4aa449981f74aff39e9b70a82a10ed5f615e41f7fb847905641ecc57.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-large-cnn/config.json not found in cache or force_download set to True, downloading to /home/nora/.cache/torch/transformers/tmpyotgfqu4


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-large-cnn/config.json in cache at /home/nora/.cache/torch/transformers/5f0de1d2bbb8eb1a3b69656622293b3328b06b701663a9d4109359751cb4e739.a42ba2fc4aa449981f74aff39e9b70a82a10ed5f615e41f7fb847905641ecc57
INFO:transformers.file_utils:creating metadata file for /home/nora/.cache/torch/transformers/5f0de1d2bbb8eb1a3b69656622293b3328b06b701663a9d4109359751cb4e739.a42ba2fc4aa449981f74aff39e9b70a82a10ed5f615e41f7fb847905641ecc57
INFO:filelock:Lock 139862067348424 released on /home/nora/.cache/torch/transformers/5f0de1d2bbb8eb1a3b69656622293b3328b06b701663a9d4109359751cb4e739.a42ba2fc4aa449981f74aff39e9b70a82a10ed5f615e41f7fb847905641ecc57.lock
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-large-cnn/config.json from cache at /home/nora/.cache/torch/transformers/5f0de1d2bbb8eb1a3b69656622293b3328b06b701663a9d410

INFO:filelock:Lock 139864640650544 acquired on /home/nora/.cache/torch/transformers/1ae1f5b6e2b22b25ccc04c000bb79ca847aa226d0761536b011cf7e5868f0655.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-vocab.json not found in cache or force_download set to True, downloading to /home/nora/.cache/torch/transformers/tmpiyoyhr87


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-vocab.json in cache at /home/nora/.cache/torch/transformers/1ae1f5b6e2b22b25ccc04c000bb79ca847aa226d0761536b011cf7e5868f0655.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
INFO:transformers.file_utils:creating metadata file for /home/nora/.cache/torch/transformers/1ae1f5b6e2b22b25ccc04c000bb79ca847aa226d0761536b011cf7e5868f0655.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
INFO:filelock:Lock 139864640650544 released on /home/nora/.cache/torch/transformers/1ae1f5b6e2b22b25ccc04c000bb79ca847aa226d0761536b011cf7e5868f0655.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock


INFO:filelock:Lock 139862067347640 acquired on /home/nora/.cache/torch/transformers/f8f83199a6270d582d6245dc100e99c4155de81c9745c6248077018fe01abcfb.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-merges.txt not found in cache or force_download set to True, downloading to /home/nora/.cache/torch/transformers/tmpk_iz00a4


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-merges.txt in cache at /home/nora/.cache/torch/transformers/f8f83199a6270d582d6245dc100e99c4155de81c9745c6248077018fe01abcfb.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:transformers.file_utils:creating metadata file for /home/nora/.cache/torch/transformers/f8f83199a6270d582d6245dc100e99c4155de81c9745c6248077018fe01abcfb.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:filelock:Lock 139862067347640 released on /home/nora/.cache/torch/transformers/f8f83199a6270d582d6245dc100e99c4155de81c9745c6248077018fe01abcfb.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-vocab.json from cache at /home/nora/.cache/torch/transformers/1ae1f5b6e2b22b25ccc04c000bb79ca847aa226d0761536b011cf7e5868f0655.ef00af9e673c7160b4d41c

INFO:transformers.modelcard:Model card: {
  "caveats_and_recommendations": {},
  "ethical_considerations": {},
  "evaluation_data": {},
  "factors": {},
  "intended_use": {},
  "metrics": {},
  "model_details": {},
  "quantitative_analyses": {},
  "training_data": {}
}

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-large-cnn/config.json from cache at /home/nora/.cache/torch/transformers/5f0de1d2bbb8eb1a3b69656622293b3328b06b701663a9d4109359751cb4e739.a42ba2fc4aa449981f74aff39e9b70a82a10ed5f615e41f7fb847905641ecc57
INFO:transformers.configuration_utils:Model config BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": null,
  "attention_dropout": 0.0,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-large-cnn/pytorch_model.bin in cache at /home/nora/.cache/torch/transformers/821c6fa2ec26e17b4ddc9f019177b20aedee71faf1797c1c109c1e651cc93819.73d71f0899e4bd27603a3503868c9f8cf938416df2de374c864a8c3af18f981d
INFO:transformers.file_utils:creating metadata file for /home/nora/.cache/torch/transformers/821c6fa2ec26e17b4ddc9f019177b20aedee71faf1797c1c109c1e651cc93819.73d71f0899e4bd27603a3503868c9f8cf938416df2de374c864a8c3af18f981d
INFO:filelock:Lock 139862029114112 released on /home/nora/.cache/torch/transformers/821c6fa2ec26e17b4ddc9f019177b20aedee71faf1797c1c109c1e651cc93819.73d71f0899e4bd27603a3503868c9f8cf938416df2de374c864a8c3af18f981d.lock
INFO:transformers.modeling_utils:loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-large-cnn/pytorch_model.bin from cache at /home/nora/.cache/torch/transformers/821c6fa2ec26e17b4ddc9f019177b20aedee71faf1797c1c1

In [209]:
print(summarizer(text, max_length=20, min_length=5))

[{'summary_text': 'James Marshall "Jimi" Hendrix was an American rock guitarist, singer, and'}]


# Sequence Classification

In [ ]:
from transformers import pipeline

nlp = pipeline("sentiment-analysis")

print(nlp("She is a doctor"))
print(nlp("He is a doctor"))